In [113]:
# Basic libraries
import shutil 
import os
import docker
import logging
import random
import time
import concurrent.futures
from collections import defaultdict
from datetime import datetime
import csv
import yaml
import re
import pprint
import math
# Additional stuff for data handling and analysis
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# Specific libraries for machine learning
# Feature extraction and preprocessing
from sklearn.metrics.pairwise import rbf_kernel
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
# Clustering
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.cluster import AgglomerativeClustering
# Dimensionality reduction and embedding
from mvlearn.embed import KMCCA
from cca_zoo.nonparametric import KCCA
from cca_zoo.linear import CCA, MCCA
from sklearn import decomposition
from cca_zoo.preprocessing import MultiViewPreprocessing
from cca_zoo.model_selection import cross_validate
from sklearn.cross_decomposition import CCA
from cca_zoo.model_selection import GridSearchCV
# Regression based learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge

#### [Workspace]
##### (K)CCA Playground

In [4]:
# X1 = np.array([[1,2],[2,3],[3,4],[4,5]], float)
# X2 = np.array([[2,1],[3,2],[4,3],[5,4]], float)

# rng=np.random.RandomState(0)
# X = rng.random((10,5))
# Y = rng.random((10,5))

# --- Larger synthetic dataset ---
rng = np.random.RandomState(0)
X = rng.random((500, 5))
# Make Y depend on X with some nonlinear structure + noise
Y = np.sin(X[:, [0, 1]]) + 0.1 * rng.randn(500, 2)
# Pad Y to 5 dims so it matches your earlier shape
Y = np.hstack([Y, rng.random((500, 3))])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# x_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(Y_train)
# X_train_scaled = x_scaler.transform(X_train)
# X_test_scaled = x_scaler.transform(X_test)
# Y_train_scaled = y_scaler.transform(Y_train)

# --- MultiView Preprocessing (scaling both views separately) ---
preproc = MultiViewPreprocessing([StandardScaler(), StandardScaler()])
X_train_scaled, Y_train_scaled = preproc.fit_transform([X_train, Y_train])
X_test_scaled, Y_test_scaled = preproc.transform([X_test, Y_test])

# kcca = KCCA(latent_dimensions=2, kernel=["rbf", "rbf"])

kcca = KCCA(latent_dimensions=5).fit((X, Y))
score = kcca.score((X, Y))

print("KCCA score:", score)

# X_train_latent, _ = kcca.fit_transform((X_train_scaled, Y_train_scaled))
# reg = LinearRegression().fit(X_train_latent, Y_train)
# X_test_latent = kcca.transform((X_test_scaled, None))[0]
# Y_pred = reg.predict(X_test_latent)  # already in original Y-units

# cv_results = cross_validate(
#     estimator=kcca,
#     views=(X, Y),
#     cv=5,  # Number of folds
#     verbose=True
# )

cv = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {"kernel": ["poly"], "c": [[1e-1], [1e-1, 2e-1]], "degree": [[2], [2, 3]]}
kernel_reg_grid = GridSearchCV(
    KCCA(latent_dimensions=1),
    param_grid=param_grid,
    cv=cv,
    verbose=True,
    
).fit([X, Y])

# Print the best parameters and score
print("Best parameters:", kernel_reg_grid.best_params_)
print("Best cross-validation score:", kernel_reg_grid.best_score_)


# # --- 2. Kernel Ridge Regression (X → Y directly) ---
# krr = KernelRidge(kernel="rbf", alpha=1.0, gamma=0.5)  # gamma can be tuned
# krr.fit(X_train_scaled, Y_train)
# Y_pred_krr = krr.predict(X_test_scaled)

# print("True Y (test):\n", Y_test)
# print("Predicted Y (test):\n", Y_pred)
# --- Evaluate ---
# mse = mean_squared_error(Y_test, Y_pred)
# r2 = r2_score(Y_test, Y_pred)

# print("MSE:", mse)
# print("R²:", r2)
# print("True Y (first 3 rows):\n", Y_test[:3])
# print("Predicted Y (first 3 rows):\n", Y_pred[:3])

KCCA score: 2.07590903214229
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters: {'c': [0.1, 0.1], 'degree': [2, 2], 'kernel': 'poly'}
Best cross-validation score: 0.9157814302995178


#### [Processing Pipeline]
##### Read in CSV Data and format CSV Data

In [5]:
# Pipeline configurations.
RESULTS_DIR = "/usr/local/bin/results"
SCOPED_RESULTS_DIR = "./scoped_results"
CONFIG_FILE = "/usr/local/bin/scoped_results/config.yml"
FIN_CONTAINERS = "./scoped_results/died_nextflow_containers.csv"
START_CONTAINERS = "/usr/local/bin/scoped_results/started_nextflow_containers.csv"
META_DATA = "slurm-job-exporter"
DATA_SOURCE = "all"
POWER_METERING = "ebpf-mon"
POWER_STATS= "./scoped_results/task_energy_data/ebpf-mon/container_power/containers"

In [6]:
# Read in the monitoring results data.
results = "/usr/local/bin/results"
fin_containers = "/usr/local/bin/results/died_nextflow_containers.csv"
start_containers = "/usr/local/bin/results/started_nextflow_containers.csv"

for root, dirs, files in os.walk(results):
    # print(i)
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            data = pd.read_csv(file_path, index_col=0)
            # print(f"Found CSV file: {file_path}")

In [7]:
def readInResultsConf(config_file):
    """
    Read in the results configuration file and return a dictionary.
    """
    monitoring_config = config_file
    with open(monitoring_config, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)

    filtered_sources = []
    seen = set()
    for target in data['monitoring_targets'].values():
        ds = target.get('data_sources')
        if ds:
            if isinstance(ds, dict):
                ds = [ds]
            for entry in ds:
                filtered = {k: entry[k] for k in ('identifier', 'source') if k in entry}
                if (
                    'source' in filtered and
                    filtered['source'] == 'slurm-job-exporter'
                ):
                    continue
                if 'source' in filtered and 'identifier' in filtered:
                    key = (filtered['source'], filtered['identifier'])
                    if key not in seen:
                        filtered_sources.append(filtered)
                        seen.add(key)
    pprint.pprint(filtered_sources)
    return filtered_sources

filtered_sources = readInResultsConf("/usr/local/bin/results/config.yml")

[{'identifier': 'name', 'source': 'cAdvisor'},
 {'identifier': 'name', 'source': 'ebpf-mon'},
 {'identifier': 'container_name', 'source': 'docker-activity'}]


In [8]:
# Set the scope for the results data
def resultsScope(results_dir, meta_data, data_source, power_metering):
    """
   Creates a copy of the results directory and returns the cleaned file tree depending on the users scope definition.
   Meta data, data source and power metering are mandatory scope definitions.
    """
    # scoped_results_dir = shutil.copytree(results_dir, "/usr/local/bin/scoped_results", dirs_exist_ok=True)
    scoped_results_dir = shutil.copytree(results_dir, "./scoped_results", dirs_exist_ok=True)
    if data_source == 'all':
        print("Data source is set to 'all', no filtering will be applied.")
        return scoped_results_dir
    for metric in os.listdir(scoped_results_dir):
        metric_path = os.path.join(scoped_results_dir, metric)
        if not os.path.isdir(metric_path):
           continue 
    # Decide which subdir to keep for this metric
        if metric == "task_metadata":
            keep = [meta_data]
        elif metric == "task_energy_data":
            keep = [power_metering]
        else:
            keep = [data_source]
    # Walk from base dir and rm all dirs that do not match the scope and the power dirs. 
        for subdir in os.listdir(metric_path):
            subdir_path = os.path.join(metric_path, subdir)
            if os.path.isdir(subdir_path) and subdir not in keep:
                shutil.rmtree(subdir_path, ignore_errors=True)
    print("Successfully scoped results directory:", scoped_results_dir)
    return scoped_results_dir
    #         subdir_name = os.path.basename(subdir_path)
    #         # print("Sub directory name:", subdir_name)
    #         if os.path.isdir(subdir_path) and subdir_name not in [meta_data, data_source, power_metering]:
    #             shutil.rmtree(subdir_path, ignore_errors=True)
    # print("Successfully scoped results directory:", scoped_results_dir) 
    # return scoped_results_dir

scoped_results = resultsScope(RESULTS_DIR, META_DATA, DATA_SOURCE, POWER_METERING) 

Data source is set to 'all', no filtering will be applied.


In [9]:
def split_task_timeseries_by_datasource(results_dir, datasource_identifier_map, nextflow_pattern=r"nxf-[A-Za-z0-9]{23}"):
    """
    For each data source in datasource_identifier_map, traverse the results_dir,
    and for each metric, split the time series CSVs into per-task files using the correct identifier column.
    """
    for datasource, identifier in datasource_identifier_map.items():
        for root, dirs, files in os.walk(results_dir):
            if os.path.basename(root) == datasource:
                for metric in os.listdir(root):
                    metric_path = os.path.join(root, metric)
                    if os.path.isdir(metric_path):
                        containers_dir = os.path.join(metric_path, "containers")
                        os.makedirs(containers_dir, exist_ok=True)
                        for file in os.listdir(metric_path):
                            if file.endswith(".csv"):
                                file_path = os.path.join(metric_path, file)
                                df = pd.read_csv(file_path)
                                if identifier not in df.columns:
                                    print(f"Identifier '{identifier}' not found in {file_path}, skipping.")
                                    continue
                                for task_name in df[identifier].unique():
                                    if pd.isna(task_name):
                                        continue
                                    if re.match(nextflow_pattern, str(task_name)):
                                        task_df = df[df[identifier] == task_name]
                                        out_path = os.path.join(containers_dir, f"{task_name}.csv")
                                        task_df.to_csv(out_path, index=False)
                                        # print(f"Saved data for {task_name} to {out_path}")
    print("Finished splitting time series data by data source.")

datasource_identifier_map = {d['source']: d['identifier'] for d in filtered_sources}
split_task_timeseries_by_datasource(scoped_results, datasource_identifier_map)

Finished splitting time series data by data source.


In [10]:
def report_missing_tasks_all_sources(results_dir, datasource_identifier_map, fin_containers_df, container_workdirs, nextflow_pattern=r"nxf-[A-Za-z0-9]{23}"):
    """
    For each data source, report how many tasks are missing compared to the finished containers.
    """
    workdir_containers = set(container_workdirs.keys())
    for datasource, identifier in datasource_identifier_map.items():
        found_containers = set()
        for root, dirs, files in os.walk(results_dir):
            if os.path.basename(root) == datasource:
                for metric in os.listdir(root):
                    metric_path = os.path.join(root, metric)
                    if os.path.isdir(metric_path):
                        for file in os.listdir(metric_path):
                            if file.endswith(".csv"):
                                file_path = os.path.join(metric_path, file)
                                df = pd.read_csv(file_path)
                                if identifier not in df.columns:
                                    continue
                                found_containers.update(
                                    str(name) for name in df[identifier].unique()
                                    if pd.notna(name) and re.match(nextflow_pattern, str(name))
                                )
        missing_in_source = workdir_containers - found_containers
        missing_in_workdirs = found_containers - workdir_containers
        print(f"--- {datasource} ---")
        print("Containers in monitored list but NOT in", datasource + ":", missing_in_source)
        print("Count:", len(missing_in_source))
        print("Containers in", datasource, "but NOT in monitored list:", missing_in_workdirs)
        print("Count:", len(missing_in_workdirs))
        print()
        
datasource_identifier_map = {d['source']: d['identifier'] for d in filtered_sources}
fin_containers = "/usr/local/bin/results/died_nextflow_containers.csv"
fin_containers_df = pd.read_csv(fin_containers)
container_workdirs = {row['Name']: row['WorkDir'] for idx, row in fin_containers_df.iterrows()}
report_missing_tasks_all_sources(scoped_results, datasource_identifier_map, fin_containers_df, container_workdirs)

--- cAdvisor ---
Containers in monitored list but NOT in cAdvisor: {'nxf-hdpcrxCbHaOhXfMhwiyBQRwP', 'nxf-0AxXejDvdNsAUCNMiAYYZeE0', 'nxf-Wa5XoU0KrCGkA6LTS19OAJT3', 'nxf-B8O9iZnpmQrdYl0EDDc8q2hy', 'nxf-JLlNpWj0apgMVvRSd08kiS1J', 'nxf-X4F6NKQQ0VzWTEJY8etZIRey', 'nxf-AZga0I6t3dbGRnNFE9PINS6k', 'nxf-pr8TxLARUDE55ghlbSKhZ0kY', 'nxf-utHFCJlFabzcZBqEEeHm5sSl', 'nxf-iHbDmRvowiSkIBhdN2LqQXCg', 'nxf-0YdhhqVARqd70Abd75Iwp3jj', 'nxf-Ez8nwxcVyPldN0u6JLT9OP5n', 'nxf-tB1G2qmEADu0ZXgBJ00c7Ued', 'nxf-mSxI0CW4N0VEP50kAvGluwGd', 'nxf-OznnguW27zxelTG4gPM6BuF1', 'nxf-Qz9hkGvYrGs46sQhtErkXQOL', 'nxf-C0ERuuRvOTxYWtNV1pqF8GGa', 'nxf-VBLbUH5LyCbIw6T5Ag1wAgGh', 'nxf-r0rdkTsD1VRcR0RxqzAiqSnx', 'nxf-cq7RFG2XZrJMxpYDWA6iIFsl', 'nxf-nKaKMlfTitC4Jz9I9vF8HAy6', 'nxf-hhvWY3V0Xo5KxG95fBJFvs51', 'nxf-uOZMGhYEi4M6ibSFVkrtgjns', 'nxf-y2JAAwDINYyFTvPKAeEYIF4F', 'nxf-G6j8fBITBFlYxTewIjWAf2Gh', 'nxf-I2nfmMqvaElj2xS0pl5av8bb', 'nxf-pvjGCFjRfPgX8FDAnfAYcWes', 'nxf-5djexamSqHQTh1DaAiajDlt3', 'nxf-3q00GuAKOGdZlhajATtqadOY', 'nxf

In [11]:
def add_workdir_to_all_task_csvs(results_dir, container_workdirs):
    """
    For every data source and metric, update each per-task CSV in 'containers' subfolders
    with the correct WorkDir from container_workdirs.
    """
    for root, dirs, files in os.walk(results_dir):
        if os.path.basename(root) == "containers":
            for file in files:
                if file.endswith(".csv"):
                    file_path = os.path.join(root, file)
                    fin_container_df = pd.read_csv(file_path)
                    container_name = os.path.splitext(file)[0]
                    if container_name in container_workdirs:
                        workdir = container_workdirs[container_name]
                        fin_container_df['WorkDir'] = workdir
                        fin_container_df.to_csv(file_path, index=False)
                        # print(f"Updated {file_path} with work directory {workdir}")

add_workdir_to_all_task_csvs(scoped_results, container_workdirs)

In [12]:
def extract_slurm_job_metadata(slurm_metadata_path, slurm_job_col="job_name"):
    """
    Extracts slurm job metadata from time-series CSVs and writes each job's data to a separate file.
    """
    for file in os.listdir(slurm_metadata_path):
        if file.endswith("slurm_job_id.csv"):
            file_path = os.path.join(slurm_metadata_path, file)
            # print(f"Reading file: {file_path}")
            df = pd.read_csv(file_path)
            for job_name in df[slurm_job_col].unique():
                if pd.isna(job_name):
                    continue
                # print(f"Processing job: {job_name}")
                job_df = df[df[slurm_job_col] == job_name]
                out_path = os.path.join(slurm_metadata_path, f"{job_name}.csv")
                job_df.to_csv(out_path, index=False)
                # print(f"Saved data for {job_name} to {out_path}")

extract_slurm_job_metadata("/usr/local/bin/results/task_metadata/slurm-job-exporter/slurm_job_id")

In [13]:
def update_finished_containers_with_nfcore_task(slurm_metadata_path, fin_containers, workdir_col='WorkDir', slurm_workdir_col='work_dir', slurm_job_col='job_name'):
    """
    Update the finished containers file with the nf-core task name (Nextflow) by matching work directories
    with slurm job metadata.
    """

    updated = False
    for file in os.listdir(slurm_metadata_path):
        if file.endswith("slurm_job_id.csv"):
            file_path = os.path.join(slurm_metadata_path, file)
            # print(f"Reading file: {file_path}")
            df = pd.read_csv(file_path)
            fin_df = pd.read_csv(fin_containers)
            if workdir_col in fin_df.columns and slurm_workdir_col in df.columns:
                for idx, row in df.iterrows():
                    work_dir = row[slurm_workdir_col]
                    slurm_job = row[slurm_job_col]
                    if pd.isna(work_dir) or pd.isna(slurm_job):
                        print(f"Skipping row {idx} due to missing WorkDir or slurm_job.")
                        continue
                    # Update fin_df where WorkDir matches
                    fin_df.loc[fin_df[workdir_col] == work_dir, 'Nextflow'] = slurm_job
                # Write back the updated fin_df
                fin_df.to_csv(fin_containers, index=False)
                print(f"Updated {fin_containers} with slurm job info.")
                updated = True
            else:
                print("WorkDir or job_name column missing in DataFrames.")
    if not updated:
        print("No updates were made to the finished containers file.")

slurm_metadata_path = os.path.join(scoped_results, "task_metadata", "slurm-job-exporter", "slurm_job_id")
update_finished_containers_with_nfcore_task(slurm_metadata_path, FIN_CONTAINERS)

Updated ./scoped_results/died_nextflow_containers.csv with slurm job info.


In [14]:
def add_nextflow_to_all_task_csvs(results_dir, fin_containers_file, workdir_col='WorkDir', nextflow_col='Nextflow'):
    """
    For every data source and metric, update each per-task CSV in 'containers' subfolders
    with the correct Nextflow task value from the finished containers file.
    """
    fin_df = pd.read_csv(fin_containers_file)
    # Ensure WorkDir is string and stripped in fin_df
    fin_df[workdir_col] = fin_df[workdir_col].astype(str).str.strip()
    for root, dirs, files in os.walk(results_dir):
        if os.path.basename(root) == "containers":
            for file in files:
                if file.endswith(".csv"):
                    file_path = os.path.join(root, file)
                    container_df = pd.read_csv(file_path)
                    if workdir_col in container_df.columns:
                        # Ensure WorkDir is string and stripped in container_df
                        container_df[workdir_col] = container_df[workdir_col].astype(str).str.strip()
                        workdir = container_df[workdir_col].iloc[0]
                        match = fin_df[fin_df[workdir_col] == workdir]
                        if not match.empty and nextflow_col in match.columns:
                            nextflow_value = match[nextflow_col].values[0]
                            container_df[nextflow_col] = nextflow_value
                            container_df.to_csv(file_path, index=False)
                            # print(f"Updated {file_path} with Nextflow value {nextflow_value}")
                        else:
                            print(f"No matching Nextflow value found for WorkDir {workdir} in {file_path}") 

add_nextflow_to_all_task_csvs(scoped_results, FIN_CONTAINERS)

No matching Nextflow value found for WorkDir /storage/nf-core/exec/work/58/b52bef7fc95a4a579c7383476e6aae in ./scoped_results/task_memory_data/cAdvisor/container_memory_usage_bytes/containers/nxf-bEdYAgPSeAi8wXfUSgSNRVZn.csv
No matching Nextflow value found for WorkDir /storage/nf-core/exec/work/a7/404a25265432bb9f8f9bb99f462123 in ./scoped_results/task_memory_data/cAdvisor/container_memory_usage_bytes/containers/nxf-lIjuTGpZakXR7QSqkIOKbZoX.csv
No matching Nextflow value found for WorkDir /storage/nf-core/exec/work/fc/b39bf10d7ddb8068491832d60da327 in ./scoped_results/task_memory_data/cAdvisor/container_memory_usage_bytes/containers/nxf-WmrVWwD6tZ0ReLKDwuqF0kTo.csv
No matching Nextflow value found for WorkDir /storage/nf-core/exec/work/58/65457826cccaa35805374aa488368a in ./scoped_results/task_memory_data/cAdvisor/container_memory_usage_bytes/containers/nxf-eGWq7LRmqmNv39ec2O2uOhMY.csv
No matching Nextflow value found for WorkDir /storage/nf-core/exec/work/b2/0ac73af0c581f25e342250f97

##### Feature Extraction
##### Build feature vectors for every observed task.
Use the raw time-series, smooth the data and and for equal length vectors.

In [ ]:
def build_container_temporal_signatures_scoped_sources(results_dir, fin_containers_file):
    """
    Build feature vectors for the scoped data sources and metrics by scanning every containers directory
    under every metric for every data source. Returns a dictionary of container temporal signatures.
    As the power consumption data of the workflow tasks will be used as labels to train models, it will be excluded from the temporal signatures.
    Each container will have a 'temporal_signatures' dict with keys like 'source/metric' for every metric from the scoped data source(s).
    """
    # Takes container name
    # df = pd.read_csv(fin_containers_file)
    # pattern_temporal_signatures = {}
    # for idx, row in df.iterrows():
    #     pattern_temporal_signatures[row['Name']] = {
    #         'temporal_signatures': {
    #         }
    #     }


    # Takes Nextflow task name
    df = pd.read_csv(fin_containers_file)
    pattern_temporal_signatures = {}
        
    for idx, row in df.iterrows():
        if  row['Nextflow'] != '':
            pattern_temporal_signatures[row['Nextflow']] = {
                'temporal_signatures': {}
            }
        else:
            continue
    # pprint.pprint(pattern_temporal_signatures)

    # Keep track of the longest time-series in the dataset to decide on truncation/padding length if needed.
    # max_length = 0
    # for root, dirs, files in os.walk(results_dir):
    #     if os.path.basename(root) == "containers":
    #         for file in files:
    #             if file.endswith(".csv"):
    #                 file_path = os.path.join(root, file)
    #                 ts_container_df = pd.read_csv(file_path)
    #                 ts_container_df['timestamp'] = pd.to_datetime(ts_container_df['timestamp'], unit='ns')
    #                 ts_container_df.set_index('timestamp', inplace=True)
    #                 value_cols = [col for col in ts_container_df.columns if col.startswith('Value')]
    #                 if not value_cols:
    #                     continue
    #                 resource_series = ts_container_df[value_cols[0]]
    #                 max_length = max(max_length, len(resource_series))
    
    # Feature vectors
    for root, dirs, files in os.walk(results_dir):
        if "task_energy_data" in root.split(os.sep):
            continue
        if "task_" in os.path.basename(root):
            workload_name = os.path.basename(root)
            print("Current workload:", workload_name)
        if os.path.basename(root) == "containers":
            metric_name = os.path.basename(os.path.dirname(root))
            for file in files:
                if file.endswith(".csv"):
                    file_path = os.path.join(root, file)
                    ts_container_df = pd.read_csv(file_path)
                    if 'Nextflow' in ts_container_df.columns:
                        task_name = ts_container_df['Nextflow'].iloc[0]
                        if task_name is not None and pd.isna(task_name):
                            print(f"Nextflow task name missing in {file_path}, skipping.")
                            continue
                    else:
                        print(f"'Nextflow' column missing in {file_path}, skipping.")
                        continue
                    # print(f"Processing task and file :", {task_name}, {file})
                    ts_container_df['timestamp'] = pd.to_datetime(ts_container_df['timestamp'], unit='ns')
                    ts_container_df.set_index('timestamp', inplace=True)
                    value_cols = [col for col in ts_container_df.columns if col.startswith('Value')]
                    if not value_cols:
                        continue
                    resource_series = ts_container_df[value_cols[0]]  

                    # Feature extraction

                    # This only takes 10 evenly spaced samples from the time series as a simple pattern representation.
                    pattern_vector = resource_series.iloc[np.round(np.linspace(0, len(resource_series) - 1, 10)).astype(int)].to_numpy()

                    # replace each point by the Gaussian-weighted mean of the surrounding 6 samples (≈3 s window, std=2 points), drops the initial NaNs, and outputs the resulting smoothed values
                    # window=6 sets the smoothing scale to 3 s at 500 ms sampling, while std=2 makes the Gaussian weight concentrate on the central few points but still include the full window, yielding features that emphasize short-term local patterns without being dominated by noise
                    # pattern_vector = resource_series.rolling(window=6, win_type='gaussian').mean(std=2).dropna().to_numpy()

                    # # PPA 
                    # n_segments = 10  # Define the number of segments
                    # segment_size = len(resource_series) // n_segments

                    # Truncate the series to make it divisible by the number of segments
                    # truncated_series = resource_series[:segment_size * n_segments]

                    # Reshape the series into segments and calculate the mean of each segment
                    # segment_vector = truncated_series.values.reshape(n_segments, segment_size).mean(axis=1)
                    

                    # Truncate the series or pad them to fixed length if intra-task lenght variability is too high
                    # pattern_vector = np.pad(pattern_vector, (0, max(0, max_length - len(pattern_vector))), mode='constant')

                    # server_spec = {
                    #     'GHz x Cores': "",
                    #     'GFlops': "",
                    #     'RAM': "",
                    #     'IOPS': "",
                    #     'Max Network Throughput': "",
                    # }
                    
                    feature_vector = { 
                        'pattern' : pattern_vector
                    }

                    # container_name = os.path.splitext(file)[0]
                    if task_name in pattern_temporal_signatures:
                        # Validation step to account for missing feature values
                        if feature_vector is not None and feature_vector != {}:
                            expected_keys = ['pattern']
                            missing_values = [key for key in expected_keys if key not in feature_vector or feature_vector[key] is None]
                            if missing_values:
                                print(f"Warning: Missing values in feature vector for {metric_name}: {missing_values}")
                            if 'pattern_vector' in feature_vector:
                                if not isinstance(feature_vector['pattern_vector'],np.ndarray):
                                    print(f"WARNING: {metric_name} pattern_vector shape: {feature_vector['pattern_vector'].shape}")
                            if workload_name not in pattern_temporal_signatures[task_name]['temporal_signatures']:
                                pattern_temporal_signatures[task_name]['temporal_signatures'][workload_name] = {} 
                            pattern_temporal_signatures[task_name]['temporal_signatures'][workload_name][metric_name] = feature_vector
    pprint.pprint(pattern_temporal_signatures)
    return pattern_temporal_signatures

task_pattern_temporal_signatures = build_container_temporal_signatures_scoped_sources(scoped_results, FIN_CONTAINERS)

Current workload: task_network_data
'Nextflow' column missing in ./scoped_results/task_network_data/cAdvisor/container_network_transmit_bytes_total/containers/PEAK_Series_nxf-sYRw50ZGcahwGZ1cClf2E9i7.csv, skipping.
'Nextflow' column missing in ./scoped_results/task_network_data/cAdvisor/container_network_transmit_bytes_total/containers/nxf-O2dqsJTBsEpCoVmCuwdUUXrK.csv, skipping.
'Nextflow' column missing in ./scoped_results/task_network_data/cAdvisor/container_network_transmit_bytes_total/containers/nxf-9OkZbAbZe3EVDUt83hyQ41ay.csv, skipping.
'Nextflow' column missing in ./scoped_results/task_network_data/cAdvisor/container_network_transmit_bytes_total/containers/PEAK_Series_nxf-aH84jeLVixOEyxqi95sYJDD9.csv, skipping.
'Nextflow' column missing in ./scoped_results/task_network_data/cAdvisor/container_network_transmit_bytes_total/containers/PEAK_Series_nxf-400sWIGEqpeJvCmEkkHqd3WL.csv, skipping.
'Nextflow' column missing in ./scoped_results/task_network_data/cAdvisor/container_network_tr

In [16]:
def cleanFeatureVectors(task_temporal_signatures):
    """
    Clean the feature vectors by removing containers that have no temporal signatures.
    This function modifies the input dictionary in place.
    Works with nested structure: {'container': {'temporal_signatures': {'workload': {'metric': {...}}}}}
    """
    cleaned_task_temporal_signatures = task_temporal_signatures.copy()
    none_counter = 0
    to_delete = []
    for name, info in cleaned_task_temporal_signatures.items():
        if not info['temporal_signatures']:
            none_counter += 1
            to_delete.append(name)
    print(f"Total containers with no signature for any metric: {none_counter}")

    for name in to_delete:
        del cleaned_task_temporal_signatures[name]

    print(f"Remaining containers after cleaning: {len(cleaned_task_temporal_signatures)}")

    # Collect all (workload, metric) pairs present in the data
    all_workloads = set()
    all_metrics = set()
    all_pairs = set()
    for info in cleaned_task_temporal_signatures.values():
        for workload, metrics in info['temporal_signatures'].items():
            all_workloads.add(workload)
            for metric in metrics.keys():
                all_metrics.add(metric)
                all_pairs.add((workload, metric))
    all_workloads = sorted(all_workloads)
    all_metrics = sorted(all_metrics)
    all_pairs = sorted(all_pairs)
    print(f"All workloads found: {all_workloads}")
    print(f"All metrics found: {all_metrics}")

    all_feature_names = set()
    for info in cleaned_task_temporal_signatures.values():
        for workload_metrics in info['temporal_signatures'].values():
            for metric in workload_metrics.values():
                all_feature_names.update(metric.keys())
    all_feature_names = sorted(all_feature_names)

    containers_with_all_pairs = []
    for container, info in cleaned_task_temporal_signatures.items():
        container_pairs = set()
        for workload, metrics in info['temporal_signatures'].items():
            for metric in metrics.keys():
                container_pairs.add((workload, metric))
        if container_pairs == set(all_pairs):
            containers_with_all_pairs.append(container)
    print(f"Keeping {len(containers_with_all_pairs)} containers with all workload/metric pairs.")

    # Filtered dict: only containers in containers_with_all_pairs
    filtered_containers_temporal_signatures = {
        k: v for k, v in cleaned_task_temporal_signatures.items()
        if k in containers_with_all_pairs
    }

    return (
        containers_with_all_pairs,
        all_pairs,
        all_feature_names,
        filtered_containers_temporal_signatures,
        all_metrics
    )
tasks_with_all_pairs, all_pairs, all_feature_names, filtered_tasks_temporal_signatures, all_metrics = cleanFeatureVectors(task_pattern_temporal_signatures)
pprint.pprint(filtered_tasks_temporal_signatures)

Total containers with no signature for any metric: 12
Remaining containers after cleaning: 384
All workloads found: ['task_cpu_data', 'task_disk_data', 'task_memory_data']
All metrics found: ['container_blkio_device_usage_total', 'container_cpu_user_seconds_total', 'container_fs_io_current', 'container_fs_reads_bytes_total', 'container_fs_writes_bytes_total', 'container_memory_usage_bytes', 'container_weighted_cycles', 'cpuPercent', 'memoryUsage']
Keeping 125 containers with all workload/metric pairs.


{'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_ALIGN_BWA_BAM_SORT_STATS_SAMTOOLS_BAM_STATS_SAMTOOLS_SAMTOOLS_FLAGSTAT_(GM12878_OMNI_REP1_T1)': {'temporal_signatures': {'task_cpu_data': {'container_cpu_user_seconds_total': {'pattern': array([ 0.576834,  5.770177,  9.85363 , 15.593169, 20.775473, 26.602281,
       30.604222, 31.972081, 31.972081, 31.972081])},
                                                                                                                                                                             'container_weighted_cycles': {'pattern': array([1.02092048e+15, 1.02092048e+15, 1.02092048e+15, 1.02092048e+15,
       1.02092048e+15, 1.02092048e+15, 1.02092048e+15, 1.02092048e+15,
       1.02092048e+15, 1.02092048e+15])},
                                                                                                                                                                             'cpuPercent': {'pattern': array([0.035586, 0.035741, 0.036343, 0.030699, 0.032266

In [201]:
def buildFeatureMatriceInput(tasks_with_all_pairs, filtered_tasks_temporal_signatures):
    """
    Build the feature matrices for the containers with all metrics and all workloads.
    Returns the feature matrix and the container names.
    """
    # Collect all (workload, metric, feature) triplets present in the data
    all_triplets = set()
    for info in filtered_tasks_temporal_signatures.values():
        for workload, metrics in info['temporal_signatures'].items():
            for metric, feats in metrics.items():
                for feat in feats.keys():
                    all_triplets.add((workload, metric, feat))
    all_triplets = sorted(all_triplets)

    # Build full feature names
    full_feature_names = [f"{w}_{m}_{f}" for (w, m, f) in all_triplets]
    
    # print(f"Total features: {full_feature_names}")
    
    feature_matrix_x = []
    task_names_x = []
    for task in tasks_with_all_pairs:
        info = filtered_tasks_temporal_signatures[task]
        row = []
        for workload, metric, feat in all_triplets:
            value = (
                info['temporal_signatures']
                .get(workload, {})
                .get(metric, {})
                .get(feat, None)
            )
            if isinstance(value, np.ndarray):
                row.extend(value.tolist())
            else:
                row.append(value)
        feature_matrix_x.append(row)
        task_names_x.append(task)

        
    feature_matrix_x = np.array(feature_matrix_x)
    print(f"Feature matrix shape: {feature_matrix_x.shape}")
    df = pd.DataFrame(feature_matrix_x)
    print(df)
    return feature_matrix_x, full_feature_names, task_names_x

# With pattern temporal signatures
feature_matrix_x, full_feature_names, task_names_x = buildFeatureMatriceInput(
    tasks_with_all_pairs, filtered_tasks_temporal_signatures
)

Feature matrix shape: (125, 90)
           0            1             2             3             4   \
0    4.691082     6.330613      8.130447     14.824244     16.698624   
1    5.203944    10.742457     16.407363     23.154555     30.478400   
2    0.027556   155.584826    319.889548    472.804563    631.453170   
3    0.023701   174.386086    363.448590    557.709471    733.940781   
4    0.036066   183.503971    464.264476    745.020157   1018.439945   
..        ...          ...           ...           ...           ...   
120  0.040146  5147.404623  10384.338499  15623.452344  20854.758146   
121  0.047363  4798.716995   9687.569400  14563.786241  19446.728700   
122  0.053844   635.764875   1406.216327   2161.070891   2916.447516   
123  0.028174   578.777020   1293.384574   2003.780385   2719.286836   
124  0.027250  4861.068343   9807.450360  14774.028048  19729.151925   

               5             6             7             8             9   \
0       16.698624     16.6

In [206]:
# Add power values from one chosen data source to all nextflow files for each data source.
# First just add the power values to fin_containers.
def addPowerToFinContainers(fin_containers, tasks_with_all_pairs, power_stats):
    """
    Add power values to the finished containers file.
    """
    
    fin_df = pd.read_csv(fin_containers)
    # power_stat_container_names = set(f[:-4] for f in os.listdir(power_stats) if f.endswith('.csv'))
    # power_stat_nextflow_names = set(
    # pd.read_csv(os.path.join(power_stats, f))['Nextflow'].iloc[0]
    # for f in os.listdir(power_stats)
    # if f.endswith('.csv') and 'Nextflow' in pd.read_csv(os.path.join(power_stats, f)).columns)

    container_to_nextflow = {}
    for f in os.listdir(power_stats):
        if f.endswith('.csv'):
            container_name = f[:-4]
            nextflow_name = pd.read_csv(os.path.join(power_stats, f))['Nextflow'].iloc[0] if 'Nextflow' in pd.read_csv(os.path.join(power_stats, f)).columns else None
            container_to_nextflow[nextflow_name] = container_name

    for task in tasks_with_all_pairs:
        power_df = pd.read_csv(os.path.join(power_stats, f"{container_to_nextflow[task]}.csv"))
        mean_power = power_df['Value (microjoules)'].mean() if 'Value (microjoules)' in power_df.columns else None
        fin_df.loc[fin_df['Name'] == container_to_nextflow[task], 'MeanPower'] = mean_power
    fin_df.to_csv(fin_containers, index=False)
    # return fin_df, container_to_nextflow
    return fin_df

# fin_df, container_to_nextflow = addPowerToFinContainers(FIN_CONTAINERS, tasks_with_all_pairs, POWER_STATS)
fin_df = addPowerToFinContainers(FIN_CONTAINERS, tasks_with_all_pairs, POWER_STATS)

### Feature Extraction for Container Runtime and Power

#### Given

- **Containers**  
  `C = {c1, c2, …, cn}`  
  *Example:* `nxf-0X0tQJagkeWOAir2jS124FfK`, `nxf-0mUZ0M8vpF30z1CEoXjCQQbH`, …

- **Metrics**  
  `M = {runtime, power}`

- **Feature Table**

| Container Name                | runtime (s) | power (μJ)      |
|-------------------------------|-------------|-----------------|
| nxf-0X0tQJagkeWOAir2jS124FfK  | 123.4       | 1.23        |
| nxf-0mUZ0M8vpF30z1CEoXjCQQbH  | 98.7        | 2.34       |

- **Features per Container**  
  `F = {runtime, power}`

##### Feature Vector

For each container `c_i`, extract:

$$
\mathbf{y}_i =
\begin{bmatrix}
\text{runtime}(c_i) \\
\text{power}(c_i)
\end{bmatrix}
$$

##### Matrix form

$$
Y =
\begin{bmatrix}
y_{1,1} & y_{1,2} \\\\
y_{2,1} & y_{2,2} \\\\
\vdots  & \vdots  \\\\
y_{n,1} & y_{n,2}
\end{bmatrix}
$$

Where each row corresponds to a container, and the columns are:
- `runtime`: execution time in seconds
- `power`: mean power consumption in microjoules

In [207]:
# Build feature output matrix for KCCA model.
def buildFeatureMatriceOutput(fin_df):
    """
    Build the feature matrices for the finished containers.
    Returns the feature matrix and the container names only for containers with available power values.
    """
    task_runtime_power = {}

    fin_df['LifeTime_s'] = (
        fin_df['LifeTime']
        .str.extract(r'([0-9.]+)(ms|s)', expand=True)
        .assign(
            value=lambda x: x[0].astype(float),
            seconds=lambda x: np.where(x[1] == 'ms', x['value'] / 1000, x['value'])
        )['seconds']
    )

    for idx, row in fin_df.iterrows():
        task_runtime_power[row['Nextflow']] = {
            'runtime': row['LifeTime_s'],
            'power': row['MeanPower']
        }
        
    feature_matrix_y = []
    task_names_y = []

    for task, info in task_runtime_power.items():
        # if container not in cleaned_container_temporal_signatures:
        
        if task not in filtered_tasks_temporal_signatures:
            continue
        if pd.notna(info['runtime']) and pd.notna(info['power']):
            feature_matrix_y.append([info['runtime'], info['power']])
            task_names_y.append(task)
            
    # Transform feature matrix K_y into numpy array
    feature_matrix_y = np.array(feature_matrix_y)
    print(f"Feature matrix shape: {feature_matrix_y.shape}")
    df = pd.DataFrame(feature_matrix_y, columns=['runtime', 'power'])

    return feature_matrix_y, task_names_y

finished_containers_dfs_with_power = addPowerToFinContainers(FIN_CONTAINERS, tasks_with_all_pairs, POWER_STATS)
filtered_fin_df = finished_containers_dfs_with_power[
    finished_containers_dfs_with_power['Nextflow'].isin(tasks_with_all_pairs)
].copy()
feature_matrix_y, task_names_y = buildFeatureMatriceOutput(filtered_fin_df)

Feature matrix shape: (115, 2)


In [209]:
# Only as workaorund if needed 
def make_same_dimension(feature_matrix_x_patterns, task_names_x, task_names_y):
    """
    Ensure that the feature matrix X and task names X only include tasks that are common with task names Y.
    """
    # Find the indices of common tasks
    common_tasks = set(task_names_x).intersection(set(task_names_y))
    indices_to_keep = [i for i, task in enumerate(task_names_x) if task in common_tasks]

    # Filter the feature matrix and task names
    feature_matrix_x_patterns = feature_matrix_x_patterns[indices_to_keep]
    task_names_x = [task for task in task_names_x if task in common_tasks]

    print(f"Filtered feature matrix shape: {feature_matrix_x_patterns.shape}")

    return feature_matrix_x_patterns, task_names_x

# Call the function
feature_matrix_x, task_names_x = make_same_dimension(feature_matrix_x, task_names_x, task_names_y)

Filtered feature matrix shape: (115, 90)


In [211]:
# Debugging output to check if the container names in X and Y match and order is the same.
container_names_x = task_names_x
container_names_y = task_names_y
print("X names:", container_names_x[:5])
print("Y names:", container_names_y[:5])
print("Length X:", len(container_names_x))
print("Length Y:", len(container_names_y))
print("All X in Y:", all(name in container_names_y for name in container_names_x))
print("All Y in X:", all(name in container_names_x for name in container_names_y))
print("Order identical:", container_names_x == container_names_y)

X names: ['nf-NFCORE_SAREK_PREPARE_REFERENCE_CNVKIT_CNVKIT_ANTITARGET_(intervals)', 'nf-NFCORE_SAREK_PREPARE_REFERENCE_CNVKIT_CNVKIT_REFERENCE_(Homo_sapiens_assembly38.fasta)', 'nf-NFCORE_SAREK_SAREK_FASTP_(HCC1395N-1)', 'nf-NFCORE_SAREK_SAREK_FASTP_(HCC1395T-1)', 'nf-NFCORE_SAREK_SAREK_FASTQC_(HCC1395N-1)']
Y names: ['nf-NFCORE_SAREK_PREPARE_REFERENCE_CNVKIT_CNVKIT_ANTITARGET_(intervals)', 'nf-NFCORE_SAREK_PREPARE_REFERENCE_CNVKIT_CNVKIT_REFERENCE_(Homo_sapiens_assembly38.fasta)', 'nf-NFCORE_SAREK_SAREK_FASTP_(HCC1395N-1)', 'nf-NFCORE_SAREK_SAREK_FASTP_(HCC1395T-1)', 'nf-NFCORE_SAREK_SAREK_FASTQC_(HCC1395N-1)']
Length X: 115
Length Y: 115
All X in Y: True
All Y in X: True
Order identical: True


#### Z-Score Transformation and Standard Scaling performed on Feature and Label Matrices

**Standard scaling** (also known as z-score normalization) is a technique used to standardize the features of a dataset so that they have the properties of a standard normal distribution with a mean of 0 and a standard deviation of 1.

##### Formula

The standard score (z-score) for a value \( x \) is calculated as:

$$
z = \frac{x - \mu}{\sigma}
$$

where:
- \( x \) is the original value,
- \( \mu \) is the mean of the training samples,
- \( \sigma \) is the standard deviation of the training samples.

This transformation is performed **independently for each feature**.
- Many machine learning algorithms assume that all features are centered around zero and have the same scale.
- Features with larger scales can dominate the objective function and negatively impact model performance.
- Standard scaling ensures that each feature contributes equally to the model.
- StandardScaler is sensitive to outliers: extreme values can affect the mean and standard deviation, leading to less robust scaling.
- For sparse data, one can disable mean centering to preserve sparsity.

#### KCCA model 

#### Basic Model fitting and results

In [90]:
# Split the data manually
X_train, X_test, Y_train, Y_test = train_test_split(feature_matrix_x, feature_matrix_y, test_size=0.3, random_state=42)

# MultiView Preprocessing 
preproc = MultiViewPreprocessing([StandardScaler(), StandardScaler()])
X_train_scaled, Y_train_scaled = preproc.fit_transform([X_train, Y_train])
X_test_scaled, Y_test_scaled = preproc.transform([X_test, Y_test])


print("X_train_scaled shape:", X_train_scaled.shape)
print("Y_train_scaled shape:", Y_train_scaled.shape)

# Find c by cv, try different kernel functions
# Define an kcca instance
kcca = KCCA(latent_dimensions=2, kernel="rbf")

# Fit the instance
kcca.fit((X_train_scaled, Y_train_scaled))

# Fit & transform
# kcca.fit_transform((X_train_scaled, Y_train_scaled))

pairwise_correlations = kcca.pairwise_correlations((X_train_scaled, Y_train_scaled))

# explained_variance = kcca.explained_variance((X_train_scaled, Y_train_scaled))
# # print("Explained variance:", explained_variance)

# explained_covariance = kcca.explained_covariance((X_train_scaled, Y_train_scaled))
# print("Explained covariance:", explained_covariance)

score = kcca.score((X_train_scaled, Y_train_scaled))

print("Score of KCCA is: ", score)

print("Pairwise correlations:", pairwise_correlations)

X_train_scaled shape: (80, 90)
Y_train_scaled shape: (80, 2)
Score of KCCA is:  1.9040062605098265
Pairwise correlations: [[[1.         1.        ]
  [0.99676333 0.90724293]]

 [[0.99676333 0.90724293]
  [1.         1.        ]]]


#### Model Selection via Hyperparameter-tuning & cross-validation

In [ ]:
# Model evaluation with cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validate(
    estimator=kcca,
    views=(X_train_scaled, Y_train_scaled),
    cv=5,  # Number of folds
    verbose=True
)

# Setting the params for the cross-validation
# param_grid = {"kernel": ["poly"], "c": [[1e-1], [1e-1, 2e-1]], "degree": [[2], [2, 3]]}
# Define the parameter grid with only the kernel parameter
param_grid = {
    "kernel": ["linear", "cosine", "rbf", "laplacian", "poly", "polynomial", "sigmoid"]
}

# Perform GridSearchCV with the simplified parameter grid
kcca_grid = GridSearchCV(
    KCCA(latent_dimensions=2),  # KCCA with default parameters except latent_dimensions
    param_grid=param_grid,
    cv=cv,
    verbose=True,
).fit([X_train_scaled, Y_train_scaled])

# Print the best parameters and score
print("Best parameters:", kcca_grid.best_params_)
print("Best cross-validation score:", kcca_grid.best_score_)

# Applying the results
best_kcca = KCCA(
    latent_dimensions=2,
    kernel='laplacian',
)
best_kcca.fit((X_train_scaled, Y_train_scaled))

best_model_score = best_kcca.score((X_test_scaled, Y_test_scaled))
print("Best model score:", best_model_score)

best_model_pairwise_correlations = best_kcca.pairwise_correlations((X_test_scaled, Y_test_scaled))
print("Best model pairwise correlations:", best_model_pairwise_correlations)

#### Enabling view predictions from latent space X to orig. space Y

#### (1) Simple Linear Regression

In [ ]:
# TODO: I think scaling is not yet considered, therefore high errors
# Making the model predictive by regressing the latent space X to Y's space
X_train_latent, _ = kcca.fit_transform((X_train_scaled, Y_train_scaled))
reg = LinearRegression().fit(X_train_latent, Y_train)
X_test_latent = kcca.transform((X_test_scaled, None))[0]
Y_pred = reg.predict(X_test_latent)  # already in original Y-units

# Evaluate basic Linear Regression model
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("MSE:", mse)
print("R²:", r2)
print("True Y (first 3 rows):\n", Y_test[:3])
print("Predicted Y (first 3 rows):\n", Y_pred[:3])

MSE: 709.3604652334496
R²: 0.0711348588342971
True Y (first 3 rows):
 [[50.22163107 23.6611865 ]
 [16.54093242 17.21827959]
 [41.78946078  0.82303918]]
Predicted Y (first 3 rows):
 [[26.1321069  31.59974178]
 [30.61945196 15.45911612]
 [33.79652766  5.25596185]]


#### (2) Kernel Ridge Regression

In [120]:
# TODO: Why does scaling make no difference here?
# TODO: Do kernel ridge regression here
# TODO: I think scaling is not yet considered, therefore high errors
# Making the model predictive by regressing the latent space X to Y's space
X_train_latent, _ = best_kcca.fit_transform((X_train_scaled, Y_train_scaled))
# reg = LinearRegression().fit(X_train_latent, Y_train_scaled)
krr = KernelRidge().fit(X_train_latent, Y_train_scaled)
X_test_latent = best_kcca.transform((X_test_scaled, None))[0]
Y_pred_scaled = krr.predict(X_test_latent)  # already in original Y-units

# Evaluate basic Linear Regression model
mse = mean_squared_error(Y_test_scaled, Y_pred_scaled)
r2 = r2_score(Y_test_scaled, Y_pred_scaled)

print("MSE:", mse)
print("R²:", r2)
print("True Y (first 3 rows):\n", Y_test_scaled[:3])
print("Predicted Y (first 3 rows):\n", Y_pred_scaled[:3])

MSE: 0.6658403078718471
R²: 0.0906151722406745
True Y (first 3 rows):
 [[ 1.24211708e+00  8.15714770e-04]
 [-7.87898627e-01 -1.34088535e-01]
 [ 7.33890147e-01 -4.77378861e-01]]
Predicted Y (first 3 rows):
 [[-0.0405536  -0.09565387]
 [-0.07130859  0.11855431]
 [ 0.07966875 -0.82474062]]


#### CCA model 

In [127]:
# Split the data manually
X_train, X_test, Y_train, Y_test = train_test_split(feature_matrix_x, feature_matrix_y, test_size=0.3, random_state=42)

# MultiView Preprocessing 
preproc = MultiViewPreprocessing([StandardScaler(), StandardScaler()])
X_train_scaled, Y_train_scaled = preproc.fit_transform([X_train, Y_train])
X_test_scaled, Y_test_scaled = preproc.transform([X_test, Y_test])

# Find c by cv, try different kernel functions
# Define an kcca instance
mcca = MCCA(latent_dimensions=2)

# Fit the instance
mcca.fit((X_train_scaled, Y_train_scaled))

# # Fit & transform
mcca.fit_transform((X_train_scaled, Y_train_scaled))

pairwise_correlations = mcca.pairwise_correlations((X_train_scaled, Y_train_scaled))

score = mcca.score((X_train_scaled, Y_train_scaled))

print("Score of CCA is: ", score)

print("Pairwise correlations:", pairwise_correlations)

Score of CCA is:  1.8161458083519078
Pairwise correlations: [[[1.         1.        ]
  [0.9418471  0.87429871]]

 [[0.9418471  0.87429871]
  [1.         1.        ]]]


In [97]:
#  Sklearn CCA for comparison

# Split the data manually
X_train, X_test, Y_train, Y_test = train_test_split(feature_matrix_x, feature_matrix_y, test_size=0.3, random_state=42)

# MultiView Preprocessing 
preproc = MultiViewPreprocessing([StandardScaler(), StandardScaler()])
X_train_scaled, Y_train_scaled = preproc.fit_transform([X_train, Y_train])
X_test_scaled, Y_test_scaled = preproc.transform([X_test, Y_test])

# Find c by cv, try different kernel functions
# Define an kcca instance
cca = CCA(n_components=2)

# Fit the instance
cca.fit((X_train_scaled), (Y_train_scaled))

# # Fit & transform
# cca.fit_transform((X_train_scaled, Y_train_scaled))

score = cca.score(X_train_scaled, Y_train_scaled)

print("Score of CCA is: ", score)

print("Pairwise correlations:", pairwise_correlations)

Score of CCA is:  0.0008173492808516425
Pairwise correlations: [[[1.         1.        ]
  [0.99676333 0.90724293]]

 [[0.99676333 0.90724293]
  [1.         1.        ]]]


In [109]:
# TODO: I think scaling is not yet considered, therefore high errors
# Making the model predictive by regressing the latent space X to Y's space
X_train_latent, Y_train_latent = cca.fit_transform(X_train_scaled, Y_train_scaled)
reg = LinearRegression().fit(X_train_latent, Y_train_latent)
X_test_latent, _= cca.transform(X_test_scaled, Y_test_scaled)
print("X_test_latent shape:", X_test_latent.shape)
Y_pred_non_inverse = reg.predict((X_test_latent))  
print("Y_pred_non_inverse shape:", Y_pred_non_inverse.shape)
_, Y_pred_inverse = cca.inverse_transform(X_test_latent,Y_pred_non_inverse)

# Evaluate basic Linear Regression model
mse = mean_squared_error(Y_test_scaled, Y_pred_inverse)
r2 = r2_score(Y_test_scaled, Y_pred_inverse)

print("MSE:", mse)
print("R²:", r2)
print("True Y (first 3 rows):\n", Y_test_scaled[:3])
print("Predicted Y (first 3 rows):\n", Y_pred_inverse[:3])

X_test_latent shape: (35, 2)
Y_pred_non_inverse shape: (35, 2)
MSE: 1082.6170362007306
R²: -1497.4915681806444
True Y (first 3 rows):
 [[ 1.24211708e+00  8.15714770e-04]
 [-7.87898627e-01 -1.34088535e-01]
 [ 7.33890147e-01 -4.77378861e-01]]
Predicted Y (first 3 rows):
 [[  3.54088157   2.50960208]
 [ 10.76136406   7.50283281]
 [-15.87715991 -12.8642502 ]]


In [125]:
# Try the predict function of sklearn cca
X_train_latent, Y_train_latent = cca.fit_transform(X_train_scaled, Y_train_scaled)
Y_test_scaled_pred = cca.predict(X_test_scaled)

score = cca.score(X_test_scaled, Y_test_scaled)
print("Score of CCA predict is: ", score)

# print("Y_test_scaled_pred :", Y_test_scaled_pred)
# print("Y_test_scaled :", Y_test_scaled)

# Evaluate basic Linear Regression model
# mse = mean_squared_error(Y_test_scaled, Y_pred_inverse)
# r2 = r2_score(Y_test_scaled, Y_pred_inverse)

# print("MSE:", mse)
# print("R²:", r2)
# print("True Y (first 3 rows):\n", Y_test_scaled[:3])
# print("Predicted Y (first 3 rows):\n", Y_pred_inverse[:3])

Score of CCA predict is:  -0.01746621663949366


### Building the Kernel Matrix for Workflow Tasks

We build an $N \times N$ matrix $K_x$ where the $(i, j)$-th entry is the kernel evaluation $k_x(x_i, x_j)$,  
with $x_i$ and $x_j$ being the temporal signatures for tasks $i$ and $j$.

- **Each row and column** corresponds to a workflow task.
- **Each entry** $K_x[i, j] = k(x_i, x_j)$ measures the similarity between tasks $i$ and $j$ using a kernel function.
- We use the **Gaussian (RBF) kernel**, which measures similarity based on the Euclidean distance in feature space, scaled by a parameter $\sigma$.
- This kernel gives higher values when two tasks have similar temporal patterns.

### Kernel Canonical Correlation Analysis (KCCA) Overview

The **KCCA algorithm** takes the kernel matrices \( K_x \) and \( K_y \) and solves a generalized eigenvector problem. This procedure finds subspaces in the linear space spanned by the eigenfunctions of the kernel functions such that projections onto these subspaces are **maximally correlated** [7]. Traditional Canonical Correlation Analysis (CCA) aims to find useful projections of features in each view of data by computing a weighted sum. However, due to its linearity, CCA may not extract meaningful descriptors of complex data.

Kernel MCCA (KMCCA) addresses this limitation by first projecting the data into a higher-dimensional feature space **before** performing CCA in that new space.

- We refer to these projections as the **resource usage projection** and the **metric projection**, respectively.
- If the linear space associated with the Gaussian (RBF) kernel can be interpreted as clusters in the original feature space, then KCCA finds **correlated pairs of clusters** in the resource usage vector space and the performance/power vector space.

**Workflow:**
1. **Compute kernel matrices** \( K_x \) and \( K_y \) for the resource and metric features.
2. **Fit KCCA** using the training data kernel matrices.
3. **Project data** into the maximally correlated subspaces for further analysis or prediction.

Then, temporal signature of the new cluster is updated from the consolidated workloads. Such consolidation iterations stop when the clusters cannot be merged anymore since merging will incur significant interference, and/or the degradation in application performance will be intolerable.

### Clustering for Workflow Task Consolidation

Our consolidation problem can be viewed as a **clustering problem**. Traditionally, clustering algorithms group similar objects together based on a defined similarity or distance metric. However, in our context, the objective is different:

- **Goal:** Group workflow tasks that are **dissimilar** in their resource requirements.
- **Rationale:** By consolidating tasks with dissimilar resource usage, we can minimize resource contention and interference, leading to more efficient utilization of system resources.

#### Custom Distance Measure

To achieve this, we need to define a **distance measure** that captures the **interference** between the resource requirements of workflow tasks. Instead of grouping tasks with similar profiles, our distance metric should:

- Assign **larger distances** to pairs of tasks with similar resource usage (to discourage grouping them together).
- Assign **smaller distances** to pairs of tasks with complementary or non-overlapping resource usage (to encourage their consolidation).

In [129]:
def compute_nextflow_task_peak_series(results_dir):
    """
    For every data source and metric, update each per-task CSV in 'containers' subfolders
    with the correct Nextflow task value from the finished containers file.
    """
    
    for root, dirs, files in os.walk(results_dir):
        if os.path.basename(root) == "containers":
            for file in files:
                if file.endswith(".csv"):
                    file_path = os.path.join(root, file)
                    task_df = pd.read_csv(file_path)
                    task_df['timestamp'] = pd.to_datetime(task_df['timestamp'], unit='ns')
                    task_df.set_index('timestamp', inplace=True)
                    value_cols = [col for col in task_df.columns if col.startswith('Value')]
                    if not value_cols:
                        # print(f"Skipping {file_path} as it does not contain 'value' column.")
                        continue
                    resource_series = task_df[value_cols[0]]
                    # print(f"Processing {file_path} with resource series: {resource_series.name}")
                    # Compute the peak series
                    peak_series = resource_series.resample('3s').max()
                    peak_df = peak_series.reset_index()
                    # print(peak_series.head())
                    peak_df.columns = ['timestamp','peak_value']
                    out_file = os.path.join(root, f"PEAK_Series_{file}")
                    peak_df.to_csv(out_file, index=False)
                    # print(f"Updated {file_path} with peak series for distance calculation in {out_file}")
    return scoped_results

compute_nextflow_task_peak_series(scoped_results)

'./scoped_results'

In [ ]:
# Some ideas on how to handle the raw peak time series data for the workload types.
def normalizePeakTimeSeries(df):
    """
    Normalize the peak time series by scaling the 'peak_value' column.
    """

    df = df.copy()
    df['relative_time'] = (pd.to_datetime(df['timestamp']) - pd.to_datetime(df['timestamp']).iloc[0]).dt.total_seconds()

    return df

def interpolatePeakTimeSeries(df, n_points=100):
    df = df.copy()
    # Ensure rel_time is sorted
    df = df.sort_values('relative_time')
    # Interpolate peak_value to n_points
    interp_times = np.linspace(df['relative_time'].min(), df['relative_time'].max(), n_points)
    interp_values = np.interp(interp_times, df['relative_time'], df['peak_value'])
    return interp_times, interp_values

def truncatePeakTimeSeries(df_i, df_j):
    """
    Truncate the peak time series to the length of the shorter series.
    """
    if len(df_i) == len(df_j):
        print("Both series are of equal length:", len(df_i))
        return df_i, df_j
    min_length = min(len(df_i), len(df_j))
    df_i = df_i.iloc[:min_length]
    df_j = df_j.iloc[:min_length]
    print("Truncated series to length:", min_length)
    return df_i, df_j

def truncateTaskInput(filtered_tasks_temporal_signatures, n=40):
    # Select n random keys to keep
    keep_keys = random.sample(list(filtered_tasks_temporal_signatures.keys()), n)
    # Build a new dict with only those keys
    shortened_filtered_tasks_temporal_signatures = {k: filtered_tasks_temporal_signatures[k] for k in keep_keys}
    return shortened_filtered_tasks_temporal_signatures

shortened_filtered_tasks_temporal_signatures = truncateTaskInput(filtered_tasks_temporal_signatures, n=40)
pprint.pprint(shortened_filtered_tasks_temporal_signatures)
    

If two tasks have very similar peak patterns, then the correlation terms are high, the product is large, and the resulting distance value is large. That means they are “similar tasks” in terms of contention risk.

If two tasks have uncorrelated or negatively correlated peaks, the correlation terms are low or near zero, so the product is small and the summed distance is small. That corresponds to “dissimilar tasks” that use resources at different times or in different ways, i.e. good consolidation candidates.

In [ ]:
# Helper to get the according peak time series for the current nextflow task.
def getPeakTimeSeriesForTask(task_name, scoped_results, type = None):
    """
    Get the peak time series for a given task name.
    """
    
    inverted_workload_type = next((k for k, v in workload_type_map.items() if v == type), None)

    current_workload_dir = os.path.join(scoped_results, inverted_workload_type) if inverted_workload_type else scoped_results

    for root, dirs, files in os.walk(current_workload_dir):
        if os.path.basename(root) == "containers":
            peak_file = os.path.join(root, f"PEAK_Series_{task_name}.csv")
            if os.path.exists(peak_file):
                if type is not None:
                    print(f"Found peak time series file for {task_name} with workload type {type}") 
                return pd.read_csv(peak_file)
    print(f"Peak time series file not found for task: {task_name}")
    return None

# To get affinity score for a pair:
def get_affinity_score(type1, type2, aff_df):
    # Try both (type1, type2) and (type2, type1) for symmetry
    row = aff_df[
        ((aff_df['workload_1'] == type1) & (aff_df['workload_2'] == type2)) |
        ((aff_df['workload_1'] == type2) & (aff_df['workload_2'] == type1))
    ]
    if not row.empty:
        return row['affinity_score'].values[0]
    else:
        return None


# Correlation can be NaN if two compared time series have no overlapping timestamps or if one of them has constant values.
# For some containers one time disk reads or for short lived container the memory consumption is constant
# which hinders the correlation calculation.
# delete this task with constant peak series from the distance matrix
def computeTaskSignatureDistances(scoped_results, filtered_tasks_temporal_signatures, container_to_nextflow):
    """
    Compute the distances between task signatures in the feature space.
    Returns a distance matrix based on the custom distance function.
    
    Args:
        scoped_results: Result dictionary holding the peak time series for each task's metric.
    Returns:
        distance_matrix: Numpy array of distances between task signatures.
    """
    
    # Get the affinity scores of the workload experiments
    aff_df = pd.read_csv("affinity_score_matrix.csv")
    
    # Use the keys of cleaned_container_temporal_signatures as task identifiers
    nextflow_jobs = list(filtered_tasks_temporal_signatures.keys())
    
    filtered_jobs = []
    for job in nextflow_jobs:
        # print(f"Processing job: {job}")
        peak_df = getPeakTimeSeriesForTask(container_to_nextflow[job], scoped_results, container_to_nextflow)
        if peak_df is not None and not peak_df['peak_value'].nunique() == 1:
            filtered_jobs.append(job)

    distance_matrix = np.full((len(filtered_jobs), len(filtered_jobs)), np.nan)
    
    # Catch the calculated distances for the job pair i,j for distribution mapping
    distances = []

    for i in range(len(filtered_jobs)):
        for j in range(i + 1, len(filtered_jobs)):
            job_i = filtered_jobs[i]
            job_j = filtered_jobs[j]
            workloads_i = list(filtered_tasks_temporal_signatures[job_i]['temporal_signatures'].keys())
            workloads_j = list(filtered_tasks_temporal_signatures[job_j]['temporal_signatures'].keys())

            # Reset the temporary terms for each job pair
            distance_i_j = 0.0
            print("Reset distance for next job pair:", job_i, job_j)
            
            # Keep track of processed affinity pairs per task
            processed_pairs = set()
            for wi in workloads_i:
                for wj in workloads_j:
                    type_i = workload_type_map.get(wi, wi)
                    type_j = workload_type_map.get(wj, wj)

                    pair = frozenset([type_i, type_j])
                    if pair in processed_pairs:
                        continue
                    processed_pairs.add(pair)

                    # -------------------------------------------------------------------------------------------------
                    # TERM 1 of the distance equation for each job i, j: Get the affinity score for the pair of workload types
                    # -------------------------------------------------------------------------------------------------
                    affinity_score = get_affinity_score(type_i, type_j, aff_df)
                    print(f"Processing jobs {job_i} and {job_j} with workload type {type_i} vs {type_j}: affinity_score={affinity_score}")

                    # -------------------------------------------------------------------------------------------------
                    # Term 2 of the distance equation for each job i, j: Get the peak time series of the workload type 1
                    # -------------------------------------------------------------------------------------------------
                    # If one time series is constant, set the distance to 0.
                    print("Computing correlation for workload types in TERM 2:", type_i, type_i)

                    peak_df_i = getPeakTimeSeriesForTask(container_to_nextflow[job_i], scoped_results, type_i)
                    peak_df_j = getPeakTimeSeriesForTask(container_to_nextflow[job_j], scoped_results, type_i)

                    # Truncate the peak time series in place to the same lenght
                    trun_peak_df_i, trun_peak_df_j = truncatePeakTimeSeries(peak_df_i, peak_df_j)
                    
                    # Compute the correlation for the peak time series of the same workload type
                    try:
                        corr_i_j_R1 = pearsonr(trun_peak_df_i['peak_value'], trun_peak_df_j['peak_value'])[0] 
                        if corr_i_j_R1 is None or np.isnan(corr_i_j_R1):
                            corr_i_j_R1 = 0.0
                            print(f"Setting correlation to 0 for {job_i} vs {job_j} with workload type {type_i} due to NaN value.")
                        print(f"Correlation for {job_i} vs {job_j} with workload type {type_i}: {corr_i_j_R1}")
                    except ValueError as e:
                        print(f"Error computing correlation for {job_i} vs {job_j} with workload type {type_i}{type_i}: {e}")
                        corr_i_j_R1 = 0.0
                        print(f"Setting correlation to 0 for {job_i} vs {job_j} with workload type {type_i}{type_i} due to one of two series being constant.")
                    
                    
                    # -------------------------------------------------------------------------------------------------
                    # TERM 3 of the distance equation for each job i, j: Get the correlation of the peak time series of the identical workload types 2
                    # -------------------------------------------------------------------------------------------------
                    # If one time series is constant, set the distance to 0.
                    print("Computing correlation for workload types in TERM 3:", type_j, type_j)
                    
                    peak_df_i = getPeakTimeSeriesForTask(container_to_nextflow[job_i], scoped_results, type_j)
                    peak_df_j = getPeakTimeSeriesForTask(container_to_nextflow[job_j], scoped_results, type_j)
                    
                    # Truncate the peak time series in place to the same lenght
                    trun_peak_df_i, trun_peak_df_j = truncatePeakTimeSeries(peak_df_i, peak_df_j)

                    # Compute the correlation for the peak time series of the same workload type
                    try:
                        corr_i_j_R2 = pearsonr(trun_peak_df_i['peak_value'], trun_peak_df_j['peak_value'])[0] 
                        if corr_i_j_R2 is None or np.isnan(corr_i_j_R2):
                            corr_i_j_R2 = 0.0
                            print(f"Setting correlation to 0 for {job_i} vs {job_j} with workload type {type_i} due to NaN value.")
                        print(f"Correlation for {job_i} vs {job_j} with workload type {type_j} {type_j}: {corr_i_j_R2}")
                    except ValueError as e:
                        print(f"Error computing correlation for {job_i} vs {job_j} with workload type {type_i}: {e}")
                        corr_i_j_R2 = 0.0
                        print(f"Setting correlation to 0 for {job_i} vs {job_j} with workload type {type_j} {type_j} due to one of two series being constant.")

                    # -------------------------------------------------------------------------------------------------
                    # Sum over jobs i,j per metric pair
                    # -------------------------------------------------------------------------------------------------
                    distance_i_j += affinity_score * corr_i_j_R1 * corr_i_j_R2
                
            # -------------------------------------------------------------------------------------------------
            # Write distance matrix entry for the job pair i,j
            # -------------------------------------------------------------------------------------------------
            print(f"Distance for job pair ({job_i}, {job_j}): {distance_i_j}")
            
            # Write the distances into list for distribution mapping
            distances.append(distance_i_j)
            
            distance_matrix[i, j] = distance_i_j
            # I think only one triangle of the matrix is enough. May increase performance.
            distance_matrix[j, i] = distance_i_j

    print("Distance matrix computed.")
    # Fill the diagonal with zeros (distance to self is zero)
    np.fill_diagonal(distance_matrix, 0.0)
    print("Distance matrix:\n", distance_matrix)

    distance_df = pd.DataFrame(distance_matrix, index=filtered_jobs, columns=filtered_jobs)
                    
    return distance_matrix, distances, distance_df

# Map the workload types to the affinity score matrix
workload_type_map = {
"task_memory_data": "mem",
"task_cpu_data": "cpu",
"task_disk_data": "fileio"
}
    
distance_matrix, distances, distance_df = computeTaskSignatureDistances(scoped_results, shortened_filtered_tasks_temporal_signatures, container_to_nextflow)

In [ ]:
print(distance_df.head())

The merge threshold sets the cutoff for when two tasks are considered “dissimilar enough” to be clustered together. A lower threshold means only pairs with very small distance values (i.e. highly compatible or non-interfering tasks) will be merged, leading to fewer merges and more clusters overall. A higher threshold allows merging even when tasks are less compatible, which results in more merges and fewer clusters.

You first normalize the pairwise distances with a QuantileTransformer so they follow a standard distribution. Then you look only at the unique pairwise values (lower triangle of the distance matrix) and pick a percentile cutoff. Interpreted:
Distances below this cutoff are judged “small enough” to merge.
Distances above it are kept apart.

The intuition is that the distance scale depends on the dataset (different workloads, resource ranges, correlations), so there is no universal numeric threshold. By picking a percentile, you tie the merge rule to the empirical distribution of distances in the current dataset.

Correctness comes from this: a low percentile means you only merge the most dissimilar tasks (the very smallest distances) → conservative merging, more clusters left. A higher percentile means you treat a larger fraction of tasks as “mergeable” → more aggressive merging, fewer clusters.

So the threshold is not an absolute notion of similarity but a relative cutoff within the observed distances, which makes the clustering adaptive to each dataset.

In [ ]:
def computeMergeThreshold(distance_matrix):

    # n_quantiles is set to the training set size rather than the default value
    # to avoid a warning being raised by this example
    qt = QuantileTransformer(
        n_quantiles=len(distance_matrix), output_distribution="normal" 
    )

    # transformed_distances = qt.fit_transform(np.array(distances)).reshape(-1, 1)
    transformed_distances = qt.fit_transform(distance_matrix)
    # print(transformed_distances)

    # Determine threshold
    # 1. Get the lower triangle of the distance matrix without the diagonal
    tril_values = transformed_distances[np.tril_indices_from(transformed_distances, k=-1)]
    tril_values_raw = distance_matrix[np.tril_indices_from(distance_matrix, k=-1)]

    # 2. Compute the 10th percentile
    threshold_transformed = np.percentile(tril_values, 15)
    # I only use the raw thesholds for now.
    threshold_raw = np.percentile(tril_values_raw, 40)

    print("Raw Threshold for current distance matrix:", threshold_raw)

    return threshold_raw, transformed_distances

threshold_raw, transformed_distances = computeMergeThreshold(distance_matrix)
print(distances)

Raw Threshold for current distance matrix: 64.55911147152764
[89.84315977632549, 56.55955124604755, 84.93388468580635, 46.36709705135959, 99.92649655963534, 85.9049221022842, 64.59088872172573, 31.741913791958936, 105.57118415736102, 118.18683027662075, 88.81289509020347, 48.64328763815015, 101.70268996044118, 83.90226848907537, 100.715182246745, 97.37128574567556, 100.09885865734222, 39.483329427326865, 16.56431753729463, 59.92159402689936, 25.796809428737735, 44.10692063067515, 36.45199739142219, 75.94090466529846, 105.56804142828143, 73.86054948148838, 14.287880441032133, 1.3027796112268681, 100.4487354217955, 47.53629933323687, 35.4781856540708, 85.23845834224532, 125.66760233815063, 75.41165427088353, 100.28563103210543, 45.52119829197387, 54.564821267675285, 79.19217718258855, 46.596042331167254, 140.6511570203894, 370.8411096920858, 376.10591596372836, 299.052142258657, 383.1819019884777, 24.94362076260865, 4.431886784079074, 1.9912492842870735, 22.846664404849996, 244.201017789

/home/niklas/.cache/pypoetry/virtualenvs/sharecomp-bB4WWry4-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (780) is greater than the total number of samples (40). n_quantiles is set to n_samples.
  warnings.warn(


In [154]:
# Run agglomerative clustering algorithm on the distance matrix
# TODO: Calculate the distance threshold based on distribution of the distances in the data.
def runAgglomerativeClustering(distance_matrix, threshold):
    """
    Run agglomerative clustering on the distance matrix.
    Returns the cluster labels for each task.
    
    Args:
        distance_matrix: Numpy array of distances between task signatures.
    Returns:
        cluster_labels: Numpy array of cluster labels for each task.
    """
    clustering = AgglomerativeClustering(n_clusters = None, metric='precomputed', linkage='average', compute_full_tree=True, compute_distances=False, distance_threshold=threshold).fit(distance_matrix)
    cluster_labels = clustering.labels_
    print(f"Number of clusters found: {len(set(cluster_labels))}")
    print(f"Cluster labels: {cluster_labels}")
    job_to_cluster = dict(zip(distance_matrix.index, cluster_labels))
    return cluster_labels, job_to_cluster

cluster_labels, job_to_cluster = runAgglomerativeClustering(distance_df, threshold_raw)
pprint.pprint(job_to_cluster)

Number of clusters found: 19
Cluster labels: [ 2  1  3  1 15  0 14  1  2  0  1 10 17  3  1  0 11  0 13 14  1  1  1 16
  7  1  9  6  1  6 12  4 18  6  1  0  5  2  8  1]
{'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_ALIGN_BWA_BAM_SORT_STATS_SAMTOOLS_BAM_STATS_SAMTOOLS_SAMTOOLS_FLAGSTAT_(GM12878_STD_REP2_T1)': 6,
 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_ALIGN_BWA_BAM_SORT_STATS_SAMTOOLS_SAMTOOLS_INDEX_(GM12878_STD_REP1_T1)': 1,
 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_ALIGN_BWA_BWA_MEM_(GM12878_FAST_REP2_T1)': 3,
 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_ALIGN_BWA_BWA_MEM_(GM12878_OMNI_REP1_T1)': 4,
 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_ALIGN_BWA_BWA_MEM_(GM12878_STD_REP2_T1)': 5,
 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_FASTQC_UMITOOLS_TRIMGALORE_TRIMGALORE_(GM12878_FAST_REP1_T1)': 0,
 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_FASTQC_UMITOOLS_TRIMGALORE_TRIMGALORE_(GM12878_STD_REP2_T1)': 17,
 'nf-NFCORE_ATACSEQ_ATACSEQ_MERGED_LIBRARY_BIGWIG_PLOT_DEEPTOOLS_DEEPTOOLS_COMPUTEMATRIX_REFERENCE_POINT_(GM12878_STD_REP1)': 1,
 'nf-NFCORE_ATACSEQ_AT

In [155]:
def clusterToJobs(job_to_cluster):
    """
    Convert job to cluster mapping to cluster to jobs mapping.
    Returns a dictionary where keys are clusters and values are lists of jobs in those clusters.
    """
    
    cluster_to_jobs = defaultdict(list)
    for k, v in job_to_cluster.items():
        cluster_to_jobs[v].append(k)

    # Collect keys to delete
    keys_to_delete = [k for k, v in cluster_to_jobs.items() if len(v) == 1]
    for k in keys_to_delete:
        del cluster_to_jobs[k]

    return cluster_to_jobs

cluster_to_jobs = clusterToJobs(job_to_cluster)
pprint.pprint(cluster_to_jobs)

defaultdict(<class 'list'>,
            {0: ['nf-NFCORE_CHIPSEQ_CHIPSEQ_FASTQ_FASTQC_UMITOOLS_TRIMGALORE_TRIMGALORE_(EZH2_IP_TKO_REP1_T1)',
                 'nf-NFCORE_ATACSEQ_ATACSEQ_MERGED_LIBRARY_BIGWIG_PLOT_DEEPTOOLS_DEEPTOOLS_PLOTHEATMAP_(GM12878_OMNI_REP2)',
                 'nf-NFCORE_ATACSEQ_ATACSEQ_FASTQ_FASTQC_UMITOOLS_TRIMGALORE_TRIMGALORE_(GM12878_FAST_REP1_T1)',
                 'nf-NFCORE_CHIPSEQ_CHIPSEQ_FASTQ_FASTQC_UMITOOLS_TRIMGALORE_TRIMGALORE_(FOXA1_IP_VEH_REP1_T1)',
                 'nf-NFCORE_SAREK_SAREK_BAM_VARIANT_CALLING_SOMATIC_ALL_MPILEUP_TUMOR_SAMTOOLS_MPILEUP_(HCC1395T_vs_HCC1395N)'],
             1: ['nf-NFCORE_ATACSEQ_ATACSEQ_MERGED_LIBRARY_CALL_ANNOTATE_PEAKS_MACS2_CALLPEAK_(GM12878_OMNI_REP1)',
                 'nf-NFCORE_CHIPSEQ_CHIPSEQ_FASTQ_FASTQC_UMITOOLS_TRIMGALORE_TRIMGALORE_(EZH2_IP_NTKO_REP2_T1)',
                 'nf-NFCORE_SAREK_SAREK_BAM_VARIANT_CALLING_SOMATIC_ALL_BAM_VARIANT_CALLING_SOMATIC_MANTA_MANTA_SOMATIC_(HCC1395T_vs_HCC1395N)',
      

In [146]:
def flatten_signature_dict(signature_dict):
    # signature_dict: the nested dict for one job
    flat = {}
    for workload, metrics in signature_dict.items():
        for metric, features in metrics.items():
            for feature, value in features.items():
                flat_key = f"{workload}/{metric}/{feature}"
                flat[flat_key] = value
    return flat

# TODO: Update to not only sum up the values but to weigh them with 2 factors:
# 1) The time that the jobs actually spends in his peak/min
# 2) How much the peak/min overlaps between the jobs.
def updateTaskSignatureToColoc(cluster_to_jobs, shortened_filtered_tasks_temporal_signatures):
    for k, v in cluster_to_jobs.items():
        
        # Initialize the coloc task signature
        coloc_dataframes = []
        
        for job in v:
            # print(job) 
            vector = shortened_filtered_tasks_temporal_signatures[job]['temporal_signatures']
            flattened_vector = flatten_signature_dict(vector)
            df = pd.DataFrame([flattened_vector])
            coloc_dataframes.append(df)
            
        # Merge the dataframes for the coloc task and write back to updated dict
        summed_df = pd.concat(coloc_dataframes).groupby(level=0).sum()
        print(f"Summed DataFrame for cluster {k}:\n", summed_df.head())
            
            
updateTaskSignatureToColoc(cluster_to_jobs, shortened_filtered_tasks_temporal_signatures)

Summed DataFrame for cluster 4:
                 task_memory_data/memoryUsage/pattern  \
0  [297238528.0, 1315274752.0, 300560384.0, 11789...   

  task_memory_data/container_memory_usage_bytes/pattern  \
0  [67813376.0, 1222582272.0, 291430400.0, 107302...      

  task_disk_data/container_blkio_device_usage_total/pattern  \
0  [2895872.0, 2895872.0, 2924544.0, 5218304.0, 5...          

  task_disk_data/container_fs_writes_bytes_total/pattern  \
0  [4096.0, 12288.0, 12288.0, 417792.0, 612925440...       

  task_disk_data/container_fs_reads_bytes_total/pattern  \
0  [2895872.0, 2895872.0, 2895872.0, 2912256.0, 2...      

      task_disk_data/container_fs_io_current/pattern  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                    task_cpu_data/cpuPercent/pattern  \
0  [0.045598, 0.043628, 0.041883000000000004, 0.0...   

  task_cpu_data/container_cpu_user_seconds_total/pattern  \
0  [0.468481, 35.545774, 68.889167, 103.907255, 1...       

     task_cpu_data/co

#### Trying out some examples

In [157]:
print(shortened_filtered_tasks_temporal_signatures['nf-NFCORE_SAREK_SAREK_BAM_VARIANT_CALLING_GERMLINE_ALL_BAM_VARIANT_CALLING_SINGLE_STRELKA_MERGE_STRELKA_(HCC1395N)']['temporal_signatures'])
# print(shortened_filtered_tasks_temporal_signatures['nf-NFCORE_SAREK_SAREK_FASTQ_ALIGN_BWAMEM_MEM2_DRAGMAP_SENTIEON_BWAMEM1_MEM_(HCC1395N)']['temporal_signatures'])
print(shortened_filtered_tasks_temporal_signatures['nf-NFCORE_ATACSEQ_ATACSEQ_MERGED_LIBRARY_BIGWIG_PLOT_DEEPTOOLS_DEEPTOOLS_PLOTHEATMAP_(GM12878_OMNI_REP2)']['temporal_signatures'])

{'task_memory_data': {'memoryUsage': {'pattern': array([1.76541696e+08, 3.45145344e+08, 1.62217984e+08, 1.62217984e+08,
       1.62217984e+08, 1.62217984e+08, 1.62217984e+08, 1.62217984e+08,
       1.62217984e+08, 1.62217984e+08])}, 'container_memory_usage_bytes': {'pattern': array([1.32915200e+08, 2.00896512e+08, 4.10120192e+08, 2.26111488e+08,
       2.26111488e+08, 2.26111488e+08, 2.26111488e+08, 2.26111488e+08,
       2.26111488e+08, 2.26111488e+08])}}, 'task_disk_data': {'container_blkio_device_usage_total': {'pattern': array([9236480., 9404416., 9457664., 9457664., 9457664.,       0.,
         16384.,   16384.,   16384.,   16384.])}, 'container_fs_writes_bytes_total': {'pattern': array([    0.,     0., 16384., 16384., 16384., 16384., 16384., 16384.,
       16384., 16384.])}, 'container_fs_reads_bytes_total': {'pattern': array([9236480., 9236480., 9404416., 9457664., 9457664., 9457664.,
       9457664., 9457664., 9457664., 9457664.])}, 'container_fs_io_current': {'pattern': array(

#### For some probes: Calculate the means etc. and compare them in a table against random co-locations

### Random Forest Regressor Modeling
#### Some parts of the data processing are repeated here for better understandability.

In [223]:
# Build feature output matrix with runtime labels
def buildFeatureMatriceOutput(fin_df):
    """
    Build the feature matrices for the finished containers.
    Returns the feature matrix and the container names only for containers with available power values.
    """
    task_runtime_power = {}

    fin_df['LifeTime_s'] = (
        fin_df['LifeTime']
        .str.extract(r'([0-9.]+)(ms|s)', expand=True)
        .assign(
            value=lambda x: x[0].astype(float),
            seconds=lambda x: np.where(x[1] == 'ms', x['value'] / 1000, x['value'])
        )['seconds']
    )

    for idx, row in fin_df.iterrows():
        task_runtime_power[row['Nextflow']] = {
            'runtime': row['LifeTime_s'],
            # 'power': row['MeanPower']
        }
        
    feature_matrix_y = []
    task_names_y = []

    for task, info in task_runtime_power.items():
        # if container not in cleaned_container_temporal_signatures:
        
        if task not in filtered_tasks_temporal_signatures:
            continue
        if pd.notna(info['runtime']):
            feature_matrix_y.append([info['runtime']])
            task_names_y.append(task)
            
    # Transform feature matrix K_y into numpy array
    feature_matrix_y = np.array(feature_matrix_y)
    print(f"Feature matrix shape: {feature_matrix_y.shape}")
    df = pd.DataFrame(feature_matrix_y, columns=['runtime'])

    return feature_matrix_y, task_names_y, df

finished_containers_dfs_with_power = addPowerToFinContainers(FIN_CONTAINERS, tasks_with_all_pairs, POWER_STATS)
filtered_fin_df = finished_containers_dfs_with_power[
    finished_containers_dfs_with_power['Nextflow'].isin(tasks_with_all_pairs)
].copy()
feature_matrix_y_runtime, task_names_y, df  = buildFeatureMatriceOutput(filtered_fin_df)
print(df.head())

Feature matrix shape: (125, 1)
     runtime
0   9.223425
1   0.399729
2   8.397083
3  29.695152
4  34.647605


In [224]:
# Build feature output matrix for KCCA model.
def buildFeatureMatriceOutput(fin_df):
    """
    Build the feature matrices for the finished containers.
    Returns the feature matrix and the container names only for containers with available power values.
    """
    task_runtime_power = {}

    # fin_df['LifeTime_s'] = (
    #     fin_df['LifeTime']
    #     .str.extract(r'([0-9.]+)(ms|s)', expand=True)
    #     .assign(
    #         value=lambda x: x[0].astype(float),
    #         seconds=lambda x: np.where(x[1] == 'ms', x['value'] / 1000, x['value'])
    #     )['seconds']
    # )

    for idx, row in fin_df.iterrows():
        task_runtime_power[row['Nextflow']] = {
            # 'runtime': row['LifeTime_s'],
            'power': row['MeanPower']
        }
        
    feature_matrix_y = []
    task_names_y = []

    for task, info in task_runtime_power.items():
        # if container not in cleaned_container_temporal_signatures:
        
        if task not in filtered_tasks_temporal_signatures:
            continue
        if pd.notna(info['power']):
            feature_matrix_y.append([info['power']])
            task_names_y.append(task)
            
    # Transform feature matrix K_y into numpy array
    feature_matrix_y = np.array(feature_matrix_y)
    print(f"Feature matrix shape: {feature_matrix_y.shape}")
    df = pd.DataFrame(feature_matrix_y, columns=['power'])

    return feature_matrix_y, task_names_y, df

finished_containers_dfs_with_power = addPowerToFinContainers(FIN_CONTAINERS, tasks_with_all_pairs, POWER_STATS)
filtered_fin_df = finished_containers_dfs_with_power[
    finished_containers_dfs_with_power['Nextflow'].isin(tasks_with_all_pairs)
].copy()
feature_matrix_y_power, task_names_y,df = buildFeatureMatriceOutput(filtered_fin_df)
print(df.head())

Feature matrix shape: (115, 1)
        power
0    8.102145
1    6.980731
2  106.552121
3  135.600863
4   17.218280


In [244]:
# Scale the feature matrices for regression models with runtime output labels.
def scaleFeatureMatrices(feature_matrix_x, reg_runtime_feature_matrix_y):
    """
    Scale the feature matrices using StandardScaler.
    Returns the scaled feature matrices.
    """

    # Reshape to 2D array
    reg_runtime_y = np.array(reg_runtime_feature_matrix_y)
    # print(reg_runtime_y)
    if reg_runtime_y.ndim == 1:
        reg_runtime_y = reg_runtime_y.reshape(-1,1)

    scaler_x = StandardScaler()
    scaler_y = StandardScaler()

    scaled_x = scaler_x.fit_transform(feature_matrix_x)
    scaled_y = scaler_y.fit_transform(reg_runtime_y)

    print(f"Scaled feature matrix X shape: {scaled_x.shape}")
    print(f"Scaled feature matrix Y shape: {scaled_y.shape}")
    
    return scaled_x, scaled_y, scaler_x, scaler_y

scaled_feature_matrix_x, scaled_runtime_feature_matrix_y, scaler_x, runtime_scaler_y = scaleFeatureMatrices(feature_matrix_x, feature_matrix_y_runtime)

Scaled feature matrix X shape: (115, 90)
Scaled feature matrix Y shape: (125, 1)


In [245]:
# Only as workaorund if needed 
def make_same_dimension(feature_matrix_x_patterns, task_names_x, task_names_y):
    """
    Ensure that the feature matrix X and task names X only include tasks that are common with task names Y.
    """
    # Find the indices of common tasks
    common_tasks = set(task_names_x).intersection(set(task_names_y))
    indices_to_keep = [i for i, task in enumerate(task_names_x) if task in common_tasks]

    # Filter the feature matrix and task names
    feature_matrix_x_patterns = feature_matrix_x_patterns[indices_to_keep]
    task_names_x = [task for task in task_names_x if task in common_tasks]

    print(f"Filtered feature matrix shape: {feature_matrix_x_patterns.shape}")

    return feature_matrix_x_patterns, task_names_x

# Call the function
scaled_runtime_feature_matrix_y, task_names_y = make_same_dimension(scaled_runtime_feature_matrix_y, task_names_x, task_names_y)

Filtered feature matrix shape: (115, 1)


In [246]:
# Scale the feature matrices for regression models with power output labels.
def scaleFeatureMatrices(feature_matrix_x, reg_power_feature_matrix_y):
    """
    Scale the feature matrices using StandardScaler.
    Returns the scaled feature matrices.
    """

    # Reshape to 2D array
    reg_power_y = np.array(reg_power_feature_matrix_y)
    # print(reg_power_y)
    if reg_power_y.ndim == 1:
        reg_power_y = reg_power_y.reshape(-1,1)

    scaler_x = StandardScaler()
    scaler_y = StandardScaler()

    scaled_x = scaler_x.fit_transform(feature_matrix_x)
    scaled_y = scaler_y.fit_transform(reg_power_y)

    print(f"Scaled feature matrix X shape: {scaled_x.shape}")
    print(f"Scaled feature matrix Y shape: {scaled_y.shape}")
    
    return scaled_x, scaled_y, scaler_x, scaler_y

scaled_feature_matrix_x, scaled_power_feature_matrix_y, scaler_x, reg_power_scaler_y = scaleFeatureMatrices(feature_matrix_x, feature_matrix_y_power)

Scaled feature matrix X shape: (115, 90)
Scaled feature matrix Y shape: (115, 1)


In [247]:
def splitFeatureMatrices(scaled_feature_matrix_x, scaled_power_feature_matrix_y, task_names_x, task_names_y):
    """
    Split the feature matrices into training and testing sets.
    """
    X_train, X_test, y_train_runtime, y_test_runtime, train_task_names_x, test_task_names_x, train_task_names_y, test_task_names_y = train_test_split(
        scaled_feature_matrix_x, scaled_power_feature_matrix_y, task_names_x, task_names_y, test_size=0.2, random_state=42
    )
    print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")
    return X_train, X_test, y_train_runtime, y_test_runtime, train_task_names_x, test_task_names_x, train_task_names_y, test_task_names_y

X_train, X_test, y_train_power, y_test_power, train_task_names_x, test_task_names_x, train_task_names_y, test_task_names_y = splitFeatureMatrices(scaled_feature_matrix_x, scaled_power_feature_matrix_y, task_names_x, task_names_y)
pprint.pprint(scaled_power_feature_matrix_y.shape)

# x_train_df = pd.DataFrame(X_train)
# y_train_df = pd.DataFrame(y_train_power, columns=['power'])
# print("X_train DataFrame:", x_train_df)
# x_test_df = pd.DataFrame(X_test, columns=all_metrics)
# print("X_test DataFrame:", x_test_df)

# y_train_df = pd.DataFrame(y_train_power, columns=['power'])
# print("y_train DataFrame:", y_train_df)
# y_test_df = pd.DataFrame(y_test_power, columns=['power'])
# print("y_test DataFrame:", y_test_df)

Training set shape: (92, 90), Test set shape: (23, 90)
(115, 1)


In [248]:
# Runtime
def splitFeatureMatrices(scaled_feature_matrix_x, scaled_runtime_feature_matrix_y, task_names_x, task_names_y):
    """
    Split the feature matrices into training and testing sets.
    """
    X_train, X_test, y_train_runtime, y_test_runtime, train_task_names_x, test_task_names_x, train_task_names_y, test_task_names_y = train_test_split(
        scaled_feature_matrix_x, scaled_runtime_feature_matrix_y, task_names_x, task_names_y, test_size=0.2, random_state=42
    )
    print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")
    return X_train, X_test, y_train_runtime, y_test_runtime, train_task_names_x, test_task_names_x, train_task_names_y, test_task_names_y

X_train, X_test, y_train_runtime, y_test_runtime, train_task_names_x, test_task_names_x, train_task_names_y, test_task_names_y = splitFeatureMatrices(scaled_feature_matrix_x, scaled_runtime_feature_matrix_y, task_names_x, task_names_y)
pprint.pprint(scaled_power_feature_matrix_y.shape)

# x_train_df = pd.DataFrame(X_train)
# y_train_df = pd.DataFrame(y_train_power, columns=['power'])
# print("X_train DataFrame:", x_train_df)
# x_test_df = pd.DataFrame(X_test, columns=all_metrics)
# print("X_test DataFrame:", x_test_df)

# y_train_df = pd.DataFrame(y_train_power, columns=['power'])
# print("y_train DataFrame:", y_train_df)
# y_test_df = pd.DataFrame(y_test_power, columns=['power'])
# print("y_test DataFrame:", y_test_df)

Training set shape: (92, 90), Test set shape: (23, 90)
(115, 1)


In [ ]:
# Random Forest Regressor to predict the power of colocatable tasks
def trainPowerWithRandomForest(X, y):
    """
    Train a Random Forest regressor to predict power consumption based on the feature matrix.
    """
    
    X, y = make_regression(n_features=4, n_informative=2,
                        random_state=0, shuffle=False)

    regr = RandomForestRegressor(max_depth=2, random_state=0)

    regr.fit(X, y)

    return regr


def predictPowerWithRandomForest(regressor, test_Data):
    """
    Predict the power consumption using the trained Random Forest regressor.
    
    Args:
        regressor: Trained Random Forest regressor.
        test_data: Test data for prediction.
        
    Returns:
        Predicted power consumption values.
    """

    return regressor.predict(test_Data)

# Fit the model.
trainedPredictor = trainPowerWithRandomForest(X_train, y_train_power)
# Predict power consumption for the test data.
predicted_power = predictPowerWithRandomForest(trainedPredictor, X_test)



In [ ]:
# Random Forest Regressor to predict the runtime of colocatable tasks
def trainRuntimeWithRandomForest(X, y):
    X, y = make_regression(n_features=4, n_informative=2,
                        random_state=0, shuffle=False)

    regr = RandomForestRegressor(max_depth=2, random_state=0)

    regr.fit(X, y)
    return regr
    
def predictRuntimeWithRandomForest(regressor, test_Data):
    """
    Predict the runtime using the trained Random Forest regressor.
    
    Args:
        regressor: Trained Random Forest regressor.
        test_data: Test data for prediction.
        
    Returns:
        Predicted runtime values.
    """

    return regressor.predict(test_Data)
    

# Fit the model.
trainedPredictor = trainRuntimeWithRandomForest(X_train, y_train_runtime)

# Predict the runtime for the test data.
predicted_runtime = predictRuntimeWithRandomForest(trainedPredictor, X_test)